In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%writefile .env

# TAVILY_API_KEY = your_key
# JINA_API_KEY = your_key
# CHROME_INSTANCE_PATH = /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
# BROWSER_HEADLESS=False

In [ ]:
import sys
from src.workflow import run_agent_workflow

In [ ]:
user_query = '''
   아래와 같은 requset에 대한 검토 결과를 얻고 싶어
   <request>
   다름이 아니라 담당하고 있는 고객사에서 Amazon Bedrock의 파운데이션 모델에 대해
   아래 사항에 대한 자료를 요청하였는데, 1) ~ 8)까지의 모든 항목이 아니더라도 비교 정리된, 전달 가능한 자료가 있을까요?
   
   검토 모델
   -       (AP-Northeast-2, Seoul 리전) Nova Micro, Nova Lite, Nova Pro, Claude 3.5 Sonnet, Claude 3.5 Sonnet v2, Claude 3 Haiku
   -       (US-East/West, EU-Central/North/West Region) Claude 3.7 Sonnet, (US-East/West Region) Claude 3.5 Haiku
   요청 내용
   -       모델명
   -       주요 특징
   -       입력 요금(1m토큰)
   -       출력 요금(1m토큰)
   -       컨텍스트 길이
   -       멀티 모달
   -       국내리전(또는 예상 일정) - Claude 3.7 Sonnet, Claude 3.5 Haiku에 한해서만
   -       RI 정책
   
   참고로 지난번에 유선상으로 잠깐 말씀드렸듯이 현재 KB금융그룹은 Azure 기반의 GenAI 플랫폼을 얼마 전에 오픈한 바 있는데,
   향후 추가 agent를 구성 확대할 계획이 있어 모델에 대한 검토가 이루어지는 것으로 보입니다.
   
   꼭 정리된 자료가 아니더라도 제공 가능한 내용이 있다면 공유 부탁 드립니다.
   </request>

   결과에는 거짓이 없어야해. reserch를 통해 얻은 정보를 통해서만 차트 및 도표 그리고 내용을 채워주세요.
   토큰 단위는 1M으로 해줘.
   결과는 pdf로 작성해줘
   
'''
#LG 스타일러와 삼성 제품간의 비교가 필요해. 결과는 pdf로 만들어줘
#AWS Cloud Market Analysis: Growth Prospects in Korea (2025-2030) 조사해줘. 결과는 pdf로 만들어줘, 
#research와 coder는 정말 간단하게 해줘. reporter 에이전트 테스트 중이거든.
#아주 간략하게 작성해줘.
#coder 에이전트는 사용하지 않아. 테스트 중이거든
#Transformer 알고리즘과 현 시점에서 어떤식으로 변화 발전 하는지 알려줘
#LG 스타일러의 중국 판매 방안에 대해서 알려줘
#5-7세 아동을 위한 미술학원(홈스쿨)을 운영예정이야. 나는 미술치료와 심리학을 전공했어. 이런 전문성을 접목한 브랜딩, 마케팅 방법, 운영방안, 구체적인 프로그램(월별)을 기획해줘. 그리고 브랜드 이름도 추천해줘.
# 아래와 같은 requset에 대한 검토 결과를 얻고 싶어
#    <request>
#    다름이 아니라 담당하고 있는 고객사에서 Amazon Bedrock의 파운데이션 모델에 대해
#    아래 사항에 대한 자료를 요청하였는데, 1) ~ 8)까지의 모든 항목이 아니더라도 비교 정리된, 전달 가능한 자료가 있을까요?
   
#    검토 모델
#    -       (AP-Northeast-2, Seoul 리전) Nova Micro, Nova Lite, Nova Pro, Claude 3.5 Sonnet, Claude 3.5 Sonnet v2, Claude 3 Haiku
#    -       (US-East/West, EU-Central/North/West Region) Claude 3.7 Sonnet, (US-East/West Region) Claude 3.5 Haiku
#    요청 내용
#    -       모델명
#    -       주요 특징
#    -       입력 요금(1m토큰)
#    -       출력 요금(1m토큰)
#    -       컨텍스트 길이
#    -       멀티 모달
#    -       국내리전(또는 예상 일정) - Claude 3.7 Sonnet, Claude 3.5 Haiku에 한해서만
#    -       RI 정책
   
#    참고로 지난번에 유선상으로 잠깐 말씀드렸듯이 현재 KB금융그룹은 Azure 기반의 GenAI 플랫폼을 얼마 전에 오픈한 바 있는데,
#    향후 추가 agent를 구성 확대할 계획이 있어 모델에 대한 검토가 이루어지는 것으로 보입니다.
   
#    꼭 정리된 자료가 아니더라도 제공 가능한 내용이 있다면 공유 부탁 드립니다.
#    </request>

#    결과에는 거짓이 없어야해. reserch를 통해 얻은 정보를 통해서만 차트 및 도표 그리고 내용을 채워주세요.
#    토큰 단위는 1M으로 해줘. 그리고 참고로 Nova Pro 출력 비용은 1M당 0.32 USD야 참고해
#    결과는 pdf로 작성해줘

In [ ]:
import os
import shutil

def remove_artifact_folder(folder_path="./artifacts/"):
    """
    ./artifact/ 폴더가 존재하면 삭제하는 함수
    
    Args:
        folder_path (str): 삭제할 폴더 경로
    """
    if os.path.exists(folder_path):
        print(f"'{folder_path}' 폴더를 삭제합니다...")
        try:
            # 폴더와 그 내용을 모두 삭제
            shutil.rmtree(folder_path)
            print(f"'{folder_path}' 폴더가 성공적으로 삭제되었습니다.")
        except Exception as e:
            print(f"오류 발생: {e}")
    else:
        print(f"'{folder_path}' 폴더가 존재하지 않습니다.")


#import nest_asyncio
#nest_asyncio.apply()

remove_artifact_folder()

result = run_agent_workflow(
    user_input=user_query,
    debug=False
)

# Print the conversation history
print("\n=== Conversation History ===")
print ("result", result)
for history in result["history"]:

    print ("===")
    print (f'agent: {history["agent"]}')
    print (f'message: {history["message"]}')